<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVISProject-CAValli_Team/blob/main/Preprocessing_story2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and connecting to Google Drive

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Reading the data

In [ ]:
all_files = glob.glob("/content/drive/MyDrive/DV_project/*.xlsx")
all_files.sort()
print(all_files)

['/content/drive/MyDrive/DV_project/EarlyLeavers_18_24.xlsx', '/content/drive/MyDrive/DV_project/ICT_age.xlsx', '/content/drive/MyDrive/DV_project/ICT_sex.xlsx', '/content/drive/MyDrive/DV_project/ICT_skills_obtaining.xlsx', '/content/drive/MyDrive/DV_project/InEducation.xlsx', '/content/drive/MyDrive/DV_project/NEET.xlsx', '/content/drive/MyDrive/DV_project/Skills_level.xlsx', '/content/drive/MyDrive/DV_project/Skills_sex.xlsx', '/content/drive/MyDrive/DV_project/TertiaryEducation.xlsx']


We import the 12 sheets of the NEET excel file (skipping useless rows/ columns and avoiding the print of unharmful warnings) in a dictionary of datasets named *neet_sheets*.

In [ ]:
sheet_names = ['Sheet 1', 'Sheet 2', 'Sheet 3', 'Sheet 4', 'Sheet 5', 'Sheet 6', 'Sheet 7', 'Sheet 8', 'Sheet 9', 'Sheet 10', 'Sheet 11', 'Sheet 12']
file_path = '/content/drive/MyDrive/DV_project/NEET.xlsx'
rows_to_skip = list(range(0, 9)) + [10, 11, 12] + list(range(47, 53))
selected_columns = ['TIME', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']

# Just to suppress the unharmful warnings we were getting
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category = UserWarning, module = "openpyxl")
    # Dictionary with datasets: 'neet_sheet1', ..., 'neet_sheet12'
    neet_sheets = {f'neet_sheet{i}': pd.read_excel(file_path, sheet_name = sheet_name, skiprows = rows_to_skip, na_values = ':', usecols = selected_columns)
                  for i, sheet_name in enumerate(sheet_names, start = 1)}

# Download all CSV files

We download all the CSV files we created.

In [ ]:
directory_path = '/content'
file_type = ".csv"
files_to_download = glob.glob(f"{directory_path}/*{file_type}")
zip_filename = "datasets.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in files_to_download:
        zipf.write(file, arcname = os.path.basename(file))
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>